In [15]:
import os
#import mlflow
import yaml
import time
import optuna
import psycopg2 as psycopg
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from optuna import distributions
from optuna.integration import OptunaSearchCV
from catboost import CatBoostRegressor
from sklearn.preprocessing import FunctionTransformer

TABLE_NAME = "flat_cleaned_churn"
RANDOM_STATE = 42

In [16]:
#Создаю необходимые соединения

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] =  os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# Установка URI для tracking и registry
#mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
#mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

In [25]:
EXPERIMENT_NAME = "Final prod improved model"
RUN_NAME = "Nick_project_prod_model"
REGISTRY_MODEL_NAME = 'CBR_model_prod'

In [17]:
#Загружаю необходимые артефакты.

df = pd.DataFrame(data, columns=columns)

columns_list = df.columns.tolist()
columns_text = ",".join(columns_list) 
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(columns_text)
    
df.to_csv("flat_cleaned_churn", index=False) 


In [18]:
df.head()

,id,build_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,target
0,1,16093,1994-01-01,4,55.834713,37.448383,2.64,204,17,true,16,10.10,44.799999,3,false,false,73.800003,13390000.0
1,2,3104,1960-01-01,1,55.701302,37.738918,2.80,20,5,false,1,6.00,16.500000,1,false,false,32.000000,5500000.0
2,3,11876,1977-01-01,0,55.851589,37.416008,2.48,168,12,true,9,10.30,44.000000,3,false,false,66.000000,9500000.0
3,4,9212,1970-01-01,6,55.876389,37.716415,2.64,98,9,true,5,6.50,28.000000,2,false,false,40.000000,9950000.0
4,5,22817,2014-01-01,4,55.734455,37.412422,2.64,274,10,true,9,10.08,45.720001,3,false,false,75.099998,18500000.0


In [19]:
#Функция для предобработки данных.

def transform_data(df):
    unique_counts = df.nunique()
    columns_to_drop = unique_counts[unique_counts < 2].index
    df = df.drop(columns=columns_to_drop)
    
    # Преобразование значений 'True' и 'False' в столбцах has_elevator и is_apartment в числовой формат
    df['has_elevator'] = df['has_elevator'].str.lower().replace({'false': 0, 'true': 1}).astype(int)
    df['building_type_int'] = df['building_type_int'].astype(int) #перевожу в числовой формат
    df.drop('id', axis=1, inplace=True)
    df.drop('build_id', axis=1, inplace=True)
    df.drop('is_apartment', axis=1, inplace=True)
    
    df['build_year'] = pd.to_datetime(df['build_year']).dt.year
    df['build_year'] = pd.to_numeric(df['build_year'], errors='coerce')
    
    return df

In [20]:
df = transform_data(df)

/tmp/ipykernel_27107/1559271895.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['has_elevator'] = df['has_elevator'].str.lower().replace({'false': 0, 'true': 1}).astype(int)


In [21]:
df.head()

,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,total_area,target
0,1994,4,55.834713,37.448383,2.64,204,17,1,16,10.10,44.799999,3,73.800003,13390000.0
1,1960,1,55.701302,37.738918,2.80,20,5,0,1,6.00,16.500000,1,32.000000,5500000.0
2,1977,0,55.851589,37.416008,2.48,168,12,1,9,10.30,44.000000,3,66.000000,9500000.0
3,1970,6,55.876389,37.716415,2.64,98,9,1,5,6.50,28.000000,2,40.000000,9950000.0
4,2014,4,55.734455,37.412422,2.64,274,10,1,9,10.08,45.720001,3,75.099998,18500000.0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17521 entries, 0 to 17520
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   build_year         17521 non-null  int32  
 1   building_type_int  17521 non-null  int64  
 2   latitude           17521 non-null  float64
 3   longitude          17521 non-null  float64
 4   ceiling_height     17521 non-null  float64
 5   flats_count        17521 non-null  int64  
 6   floors_total       17521 non-null  int64  
 7   has_elevator       17521 non-null  int64  
 8   floor              17521 non-null  int64  
 9   kitchen_area       17521 non-null  float64
 10  living_area        17521 non-null  float64
 11  rooms              17521 non-null  int64  
 12  total_area         17521 non-null  float64
 13  target             17521 non-null  float64
dtypes: float64(7), int32(1), int64(6)
memory usage: 1.8 MB


In [29]:
X_example = df.iloc[:50]

In [30]:
X_example.drop('target', axis=1, inplace=True)

/tmp/ipykernel_27107/1422097860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_example.drop('target', axis=1, inplace=True)


In [31]:
#Сохраню пример данных для последующего тестирования.
X_example.to_csv('/home/mle-user/mle_projects/mle-project-sprint-3-v001/services/models/X_example.csv', index=False)

In [11]:
#Закладываю функцию для категоризации по этажам

def floor_category(X, **kwargs):
    new_features = pd.DataFrame(index=X.index)
    new_features['is_first_floor'] = (X['floor'] == 1).astype(int)
    new_features['is_last_floor'] = (
        (X['floor'] == X['floors_total']).astype('int')
    )
    new_features['floor_relative'] = X['floor'] / X['floors_total']
    return new_features

custom_features_transformer = FunctionTransformer(floor_category)

In [12]:
#Формирую простой препроцессов в рамках которого будут заполнены пропущенные значения и квартиры категоризируются по этажности.

preprocessor = ColumnTransformer([
     ('fill_missing',  SimpleImputer(missing_values=0., strategy='median'), ['kitchen_area', 'living_area', 'total_area']),
     ('floor_category', FunctionTransformer(floor_category), ['floor', 'floors_total'])], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

In [13]:
#Формирую пайплайн

cbr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostRegressor(random_state=RANDOM_STATE,
                                verbose=False))
])

In [9]:
#модель без pipeline
cbr_model = CatBoostRegressor(random_state=RANDOM_STATE,
                                verbose=False)

In [32]:
#Разделяю выборку

X = df.drop('target', axis=1).copy()
y = df['target'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17521 entries, 0 to 17520
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   build_year         17521 non-null  int32  
 1   building_type_int  17521 non-null  int64  
 2   latitude           17521 non-null  float64
 3   longitude          17521 non-null  float64
 4   ceiling_height     17521 non-null  float64
 5   flats_count        17521 non-null  int64  
 6   floors_total       17521 non-null  int64  
 7   has_elevator       17521 non-null  int64  
 8   floor              17521 non-null  int64  
 9   kitchen_area       17521 non-null  float64
 10  living_area        17521 non-null  float64
 11  rooms              17521 non-null  int64  
 12  total_area         17521 non-null  float64
dtypes: float64(6), int32(1), int64(6)
memory usage: 1.7 MB


In [33]:
from catboost import CatBoostRegressor
import optuna

#Запускаю голосование
def objective(trial):
    iterations = trial.suggest_int('iterations', 600, 1200)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
    depth = trial.suggest_int('depth', 4, 10)

    model = CatBoostRegressor(iterations=iterations, learning_rate=learning_rate, depth=depth, random_state=RANDOM_STATE, verbose=False)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    error = mean_absolute_error(y_test, y_pred)

    return error

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print(study.best_params)

[I 2024-06-10 07:39:51,030] A new study created in memory with name: no-name-3776f7c1-3947-424d-81dc-726cebca7cbf
/tmp/ipykernel_4813/2958324180.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
[I 2024-06-10 07:39:55,781] Trial 0 finished with value: 1797759.146962422 and parameters: {'iterations': 702, 'learning_rate': 0.03238141057510905, 'depth': 9}. Best is trial 0 with value: 1797759.146962422.
/tmp/ipykernel_4813/2958324180.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
[I 2024-06-10 07:39:58,214] Trial 1 finished with

{'iterations': 702, 'learning_rate': 0.03238141057510905, 'depth': 9}


In [34]:
#получаю лучшие параметры
best_params = study.best_params
best_params

{'iterations': 702, 'learning_rate': 0.03238141057510905, 'depth': 9}

In [ ]:
#обучение модели на лучших гиперпараметрах.

cbr_model.set_params(**best_params)

cbr_model.fit(X_train, y=y_train)

In [36]:
#Функция для скоринга модели
def evaluate(pipeline, X_test, y_test):
    start_time = time.time()
    prediction = pipeline.predict(X_test)
    end_time = time.time()
    prediction_time = end_time - start_time
    mean = y_test.mean() 
    MAE = mean_absolute_error(y_test, prediction)
    MSE = mean_squared_error(y_test, prediction)
    R2 = r2_score(y_test, prediction)
    y_error = y_test - prediction
    y_error_abs = abs(y_error)
    perc_error_abs = y_error_abs / y_test
    mape = perc_error_abs.mean()
    
    return {
        'Среднее значение целевой переменной': mean.round(2),
        'Средний модуль ошибки': MAE.round(2),
        'Средняя квадратичная ошибка': MSE.round(2),
        'Коэффициент детерминации': R2,
        'Средняя абсолютная ошибка в процентах': mape.round(2),
        'Время предсказания в секундах': prediction_time
    }

In [37]:
sbr_score = evaluate(cbr_model, X_test, y_test)
sbr_score

{'Среднее значение целевой переменной': 11079136.06,
 'Средний модуль ошибки': 1801854.6,
 'Средняя квадратичная ошибка': 5005536981141.5,
 'Коэффициент детерминации': 0.6285431572842137,
 'Средняя абсолютная ошибка в процентах': 0.18,
 'Время предсказания в секундах': 0.004262447357177734}

In [49]:
#Kогирую модель и артефакты в MLflow

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    
pip_requirements = '../requirements.txt' 
input_example = input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(cbr_model,artifact_path="models", pip_requirements=pip_requirements,
                             input_example=input_example, metadata=metadata,
                             registered_model_name=REGISTRY_MODEL_NAME, await_registration_for=6)
    # Дополнительные логирования метрик и артефактов
    mlflow.log_metrics(sbr_score)
    mlflow.log_params(best_params) 
    mlflow.log_artifact("columns.txt", artifact_path="dataframe")
    mlflow.log_artifact("flat_cleaned_churn", artifact_path="dataframe")

/home/mle-user/mle_projects/mle-project-sprint-3-v001/.mle-project-sprint-3-v001/lib/python3.10/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/06/10 07:57:21 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (curre

In [50]:
#Проверяю доступность модели.
model_uri = "models:/CBR_model_prod/1"  # Укажите URI вашей модели
cbr_model = mlflow.sklearn.load_model(model_uri)

In [53]:
# Сохраняю модель в директории /services/models
target_dir = "/home/mle-user/mle_projects/mle-project-sprint-3-v001/services/models"

os.makedirs(target_dir, exist_ok=True)

mlflow.sklearn.save_model(cbr_model, target_dir)

print("Модель сохранена в директории:", target_dir)

Модель сохранена в директории: /home/mle-user/mle_projects/mle-project-sprint-3-v001/services/models
